In [354]:
from symmer.symplectic import (
    PauliwordOp, QuantumState, random_PauliwordOp, random_QuantumState,
    array_to_QuantumState, find_symmetry_basis
)
from symmer.chemistry import MoleculeBuilder
from symmer.utils import lp_norm, exact_gs_energy
import numpy as np

In [355]:
geometry=[
    ('H', (0.0,0.0,0.0)),
    ('H', (2.45366053071732,0.0,0.0)),
    ('H', (2.45366053071732,2.45366053071732,0.0)),
    ('H', (0.0,2.45366053071732,0.0))
     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.0	0.0
H	2.45366053071732	0.0	0.0
H	2.45366053071732	2.45366053071732	0.0
H	0.0	2.45366053071732	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.333420245395901
MP2 energy:  -1.6160819607762904
CCSD energy: -1.826156313717218
FCI energy:  -1.8743108952541816


Number of qubits: 8


In [356]:
#n_qubits = 10
#n_terms  = 100

H = molecule.H_q# random_PauliwordOp(n_qubits, n_terms, complex_coeffs=False)
print(H)
gs_nrg, gs_vec = exact_gs_energy(H.to_sparse_matrix)
print(f'Ground state energy = {gs_nrg}')
S = find_symmetry_basis(H)
S

-1.081+0.000j IIIIIIII +
 0.065+0.000j ZIIIIIII +
-0.014+0.000j YZZZZZYI +
-0.014+0.000j XZZZZZXI +
 0.065+0.000j IZIIIIII +
-0.014+0.000j IYZZZZZY +
-0.014+0.000j IXZZZZZX +
 0.038+0.000j IIZIIIII +
 0.038+0.000j IIIZIIII +
 0.038+0.000j IIIIZIII +
 0.038+0.000j IIIIIZII +
 0.014+0.000j IIIIIIZI +
 0.014+0.000j IIIIIIIZ +
 0.091+0.000j ZZIIIIII +
 0.014+0.000j YIZZZZYI +
 0.014+0.000j XIZZZZXI +
 0.019+0.000j YXXYIIII +
-0.019+0.000j YYXXIIII +
-0.019+0.000j XXYYIIII +
 0.019+0.000j XYYXIIII +
 0.057+0.000j YXIIXYII +
-0.057+0.000j YYIIXXII +
-0.057+0.000j XXIIYYII +
 0.057+0.000j XYIIYXII +
 0.014+0.000j ZYZZZZZY +
 0.014+0.000j ZXZZZZZX +
 0.024+0.000j YXIIIIXY +
-0.024+0.000j YYIIIIXX +
-0.024+0.000j XXIIIIYY +
 0.024+0.000j XYIIIIYX +
 0.050+0.000j ZIZIIIII +
 0.007+0.000j YZIZZZYI +
 0.007+0.000j XZIZZZXI +
 0.069+0.000j ZIIZIIII +
-0.027+0.000j YZZIZZYI +
-0.027+0.000j XZZIZZXI +
 0.033+0.000j YZYYZZZY +
 0.033+0.000j YZYXZZZX +
 0.033+0.000j XZXYZZZY +
 0.033+0.000j XZXXZZZX +


 1 ZIIZIZZI 
 1 IZIZIZIZ 
 1 IIZZIIII 
 1 IIIIZZII

In [363]:
gs_psi = array_to_QuantumState(gs_vec)
gs_psi.sectors_present(S)

array([ 1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j])

In [364]:
hf_psi = QuantumState([molecule.hf_array])
hf_psi.sectors_present(S)

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

Initialize the algorithm with a random state:

In [299]:
def gram_schmidt_step(v, orthogonal_set):
    u_out = v.copy()
    for u in orthogonal_set:
        u_norm = np.square(lp_norm(u.coeff_vector))
        uv_inner_prod = u.conjugate * v
        u_out -= u.multiply_by_constant(uv_inner_prod/u_norm)
    return u_out

In [365]:
v_init = random_QuantumState(H.n_qubits, n_terms=10, normalized=True)
v_init = v_init.cleanup().normalize
print(v_init)
print(v_init.conjugate * v_init)
w_init_prime = H * v_init
alpha_init = w_init_prime.conjugate * v_init
w_init = w_init_prime - v_init.multiply_by_constant(alpha_init)
print(w_init)

 0.450+0.127j |00100010> +
 0.036+0.016j |00101001> +
 0.159+0.284j |00111101> +
 0.101+0.064j |01000100> +
 0.319+0.058j |10000101> +
 0.010+0.124j |10000111> +
 0.003+0.194j |10010011> +
 0.325+0.320j |11010000> +
 0.411+0.071j |11111001> +
 0.345+0.023j |11111111>
(0.9999999999999999+0j)
-0.000-0.000j |00000101> +
 0.027+0.005j |00000111> +
 0.031+0.030j |00010011> +
-0.009-0.004j |00011010> +
 0.074+0.073j |00011100> +
-0.000-0.027j |00011110> +
 0.055+0.015j |00100010> +
-0.003-0.001j |00100110> +
-0.007-0.003j |00101001> +
-0.043-0.008j |00110100> +
 0.001+0.017j |00110110> +
-0.054-0.097j |00111101> +
 0.003+0.002j |01000100> +
-0.030-0.005j |01000110> +
-0.073-0.013j |01001001> +
-0.002-0.028j |01001011> +
-0.021-0.038j |01001111> +
 0.009+0.009j |01010010> +
-0.043-0.043j |01100001> +
-0.000-0.015j |01100011> +
-0.001-0.000j |01101000> +
 0.022+0.039j |01110011> +
 0.032+0.006j |01111011> +
-0.004-0.008j |01111100> +
-0.075-0.014j |10000101> +
-0.004-0.045j |10000111> +
 0.035

From the randomized starting point, construct the next state as follows:

In [372]:
2**8

256

In [373]:
w_current = w_init.copy()
v_current = v_init.copy()

w_list = [w_current]
v_list = [v_current]
for j in range(10):
    
    # construct v_next from w_current
    beta_next = lp_norm(w_current.coeff_vector)
    print(f'beta_next = {beta_next}')
    if beta_next>1e-5:
        v_next = w_current.multiply_by_constant(1/beta_next)
    else:
        v_next = random_QuantumState(n_qubits, n_terms=np.random.randint(0,2**n_qubits))
        v_next = gram_schmidt_step(v=v_next, orthogonal_set=v_list) # orthogonalize w.r.t to previous v vectors
    v_list.append(v_next)
    
    # construct w_next from w_current and v_next
    w_next_prime = H * v_next
    alpha_next = w_next_prime.conjugate * v_next
    w_next = w_next_prime - v_next.multiply_by_constant(alpha_next) - v_current.multiply_by_constant(beta_next)
    w_list.append(w_next)
    
    # update the current v,w vectors and repeat
    w_current = w_next
    v_current = v_next

beta_next = 0.588583462050134
beta_next = 0.7105254418298599
beta_next = 0.2933863709905624
beta_next = 0.4536454870541469
beta_next = 0.4005652019159028
beta_next = 0.2915775918836008
beta_next = 0.29988681070441375
beta_next = 0.21614697154004053
beta_next = 0.24276303846065267
beta_next = 0.29622989060824567


In [374]:
from scipy import sparse as sp

V = sp.hstack([v.to_sparse_matrix for v in v_list])

In [375]:
T = (V.H @ H.to_sparse_matrix @ V).real

In [376]:
print(abs(T.toarray())>1e-10)

[[ True  True False False False False False False False False False]
 [ True  True  True False False False False False False False False]
 [False  True  True  True False False False False False False False]
 [False False  True  True  True False False False False False False]
 [False False False  True  True  True False False False False False]
 [False False False False  True  True  True False False False False]
 [False False False False False  True  True  True False False False]
 [False False False False False False  True  True  True False False]
 [False False False False False False False  True  True  True False]
 [False False False False False False False False  True  True  True]
 [False False False False False False False False False  True  True]]


In [380]:
T_gs_nrg, T_gs_vec = exact_gs_energy(T)
print(f'Grounds state energy of H is {gs_nrg}, of T it is {T_gs_nrg}')

Grounds state energy of H is -1.8743108952541763, of T it is -1.864297630255416


In [381]:
array_to_QuantumState()

AssertionError: the statevector dimension is not a power of 2

In [382]:
T_gs_vec

array([-0.06460042,  0.11275856, -0.227759  ,  0.57423681, -0.67335141,
        0.3454961 , -0.15388286,  0.06703548, -0.02181461,  0.00792104,
       -0.00286947])

In [389]:
S_trans = (V.H @ S[0].to_sparse_matrix @ V).real

In [394]:
T_gs_vec.transpose() * S_trans.toarray() * T_gs_vec

array([[ 0.00178484, -0.00028448,  0.0218369 ,  0.10183428,  0.00786489,
        -0.00933063, -0.01045113, -0.0008357 ,  0.0000717 , -0.0000048 ,
        -0.00000063],
       [-0.00009337,  0.01153892,  0.00896264,  0.00801897,  0.07798865,
        -0.0228375 ,  0.00113767, -0.00038048, -0.00008103,  0.00000354,
         0.00000051],
       [ 0.00175675,  0.00219677,  0.02092047, -0.03730541, -0.17789592,
         0.04935879,  0.00437159,  0.00109343,  0.00005386, -0.00000303,
        -0.00000028],
       [ 0.00128879,  0.0003092 , -0.00586869, -0.03013763,  0.29039589,
        -0.01717315,  0.00830341,  0.00090409,  0.00020028, -0.00000157,
        -0.00000104],
       [ 0.00007239,  0.00218699, -0.02035324,  0.21119759, -0.01977666,
         0.05256115,  0.00063957,  0.00145569,  0.00005035,  0.00001049,
         0.00000016],
       [-0.00032621, -0.00243255,  0.0214501 , -0.04744012,  0.19964668,
         0.05833519,  0.00423705, -0.00007195, -0.0000753 ,  0.0000115 ,
         0.000